In [77]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pandas as pd 


page_number = 0
max_pages = 1
all = []  
codes = []




driver = webdriver.Chrome()


url_template = "https://www.desocialekaart.be/zoeken?pagina={}&paginaGrootte=25&rubriek=06.%20OPLEIDING,%20WERKLOOSHEID%20EN%20TEWERKSTELLING&sort=relevantie"


max_pages = 60

while page_number <= max_pages:

    url = url_template.format(page_number)
    

    driver.get(url)

  
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "soka-result__title"))
    )
    

    page_source = driver.page_source


    soup = BeautifulSoup(page_source, 'html.parser')

  
    organizations = soup.find_all("h3", class_="soka-result__title")
    for org in organizations:
        org_name = org.text
        cleaned_name = org_name.replace("businessverified", "").replace("business", "").strip()

        all.append(cleaned_name)  
    all

    postal_code_pattern = r'\b\d{4}\b'
    postal_codes = soup.find_all("p", class_="ng-star-inserted")
    for postal_code in postal_codes:
        matches = re.findall(postal_code_pattern, postal_code.text)
        if matches:
            codes.append(matches[0])


    page_number += 1 


driver.quit()




In [75]:
df = pd.DataFrame({'Name': all})
df.to_excel("result.xlsx", index=False)
